In [1]:
import sys
sys.path.append("..")

import os
import cv2
import numpy as  np
import matplotlib.pyplot as plt
from pathlib import Path
import kikuchipy as kp
from orix.quaternion import Rotation
from openECCI import io, rkp, stagecomputation
from openECCI.ctf import file_reader as ctf_reader

plt.rcParams.update({
    "figure.figsize": (7, 7),
    "font.size": 9,
})

In [2]:
optimized_correction_coefficients = {'tiltX_corr_angle': -0.5312103261068537, 'tiltY_corr_angle': 1.2285403872457656, 'tiltZ_corr_angle': 0.06380671840706255, 'PCz': 3.890243796249578}

In [3]:
# Path to Si ECP reference pattern
si_ref_ecp_path = r"D:\OneDrive - Monash University\2018-2019 MCEM\02 Projects\13 EBSD n ECCI\20231024 ECCI"
si_ref_ecp_file = r"01_Si_003_after 180 rot.tif"
si_ref_ecp_fname = os.path.join(si_ref_ecp_path, si_ref_ecp_file)
ref_ecp = plt.imread(si_ref_ecp_fname)

# Path to Si master pattern generated by EMsoft
si_MP_path = Path(r"C:\Users\IMBalENce\EMsoftData")
si_MP_file = r"Si-master-20kv.h5"
si_MP_fname = os.path.join(si_MP_path, si_MP_file)
# si_MP = kp.load(si_MP_fname, projection="lambert", hemisphere="both", energy=20)
si_ECP_MP = kp.load(si_MP_fname, projection="lambert", hemisphere="both", energy=20)

# Path to Si .ctf file containing the EBSD euler angles
si_xmap_path = r"D:\OneDrive - Monash University\2018-2019 MCEM\02 Projects\13 EBSD n ECCI\20221216 EBSD standard steel\standard steel sample 14_12_2022\standard fcc steel"
si_xmap_file = r"20kv_26nA_15mm WD_4x4 binning Si Map Data 2.ctf"
si_xmap_fname = os.path.join(si_xmap_path, si_xmap_file)

In [4]:
# Path to fcc_Fe master pattern generated by EMsoft
fe_MP_path = Path(r"D:\OneDrive - Monash University\2018-2019 MCEM\02 Projects\13 EBSD n ECCI\20230213 tutorial manuscript\supplementary")
fe_MP_file = r"Fe-master-20kV-1.h5"
fe_MP_fname = os.path.join(fe_MP_path, fe_MP_file)
fe_MP = kp.load(fe_MP_fname, projection="lambert", hemisphere="both", energy=20)

# Path to fcc_Fe .ctf file containing the EBSD euler angles
fe_xmap_path = r"D:\OneDrive - Monash University\2018-2019 MCEM\02 Projects\13 EBSD n ECCI\20230213 tutorial manuscript\supplementary"
fe_xmap_file = r"20kv_26nA_15mm WD_4x4 bin_fcc_Fe Map.ctf"
fe_xmap_fname = os.path.join(fe_xmap_path, fe_xmap_file)
fe_xmap = ctf_reader(fe_xmap_fname)
ipf_map = rkp.get_xmap_image(fe_xmap, phase_name="Iron fcc", overlay="BC").astype('float32')

# Reference Reference image or Ideal image
sem_path = r"D:\OneDrive - Monash University\2018-2019 MCEM\02 Projects\13 EBSD n ECCI\20231024 ECCI"
sem_file = r"01_steel overview.tif"
sem_fname = os.path.join(sem_path, sem_file)
sem = cv2.imread(sem_fname).astype('float32')

sem_norm = cv2.cvtColor(sem, cv2.COLOR_BGR2RGB)
sem_norm /= 255
ipf_image = cv2.cvtColor(ipf_map, cv2.COLOR_BGR2RGB)
height, width, _ = sem_norm.shape
print(height, width)

2188 3072


In [5]:
# When need to load the saved alignment points for future use, run this cell
[points1, points2] = io.load_prev_align_points()
pts_sem = np.array(points2, dtype=np.float32)
pts_ipf = np.array(points1, dtype=np.float32)

homography, mask = cv2.findHomography(pts_ipf, pts_sem, 0)
warp_src = cv2.warpPerspective(ipf_map, homography, (width, height))
inverse_transformation_matrix = np.linalg.inv(homography)

alpha = 0.6
beta = (1.0 - alpha)
dst_warp_blended = cv2.addWeighted(sem_norm, alpha, warp_src, beta, 0.0)

In [6]:
# automated correlation of ECP with low magnification SEM image
fe_ecp_path = r"D:\OneDrive - Monash University\2018-2019 MCEM\02 Projects\13 EBSD n ECCI\20231024 ECCI"
fe_ecp_file = r"steel_ECP_001.tif"
fe_ecp_fname = os.path.join(fe_ecp_path, fe_ecp_file)
fe_exp_ecp = plt.imread(fe_ecp_fname)

fe_sem_path = r"D:\OneDrive - Monash University\2018-2019 MCEM\02 Projects\13 EBSD n ECCI\20231024 ECCI"
fe_sem_file = r"steel_SEM_001.tif"
fe_sem_fname = os.path.join(fe_sem_path, fe_sem_file)
fe_exp_sem = plt.imread(fe_sem_fname)

metadata1 = io.get_sem_metadata(sem_fname)
metadata2 = io.get_sem_metadata(fe_ecp_fname)

diff_x = metadata2["stage_x"] - metadata1["stage_x"]
diff_y = metadata2["stage_y"] - metadata1["stage_y"]

diff_pix_x = diff_x / metadata1["pixel_size"]
diff_pix_y = diff_y / metadata1["pixel_size"]

centre_x = metadata1["resolution"][0]/2 + diff_pix_x
centre_y = metadata1["resolution"][1]/2 - diff_pix_y

sem_centre_point = np.array([[centre_x], [centre_y], [1]])
ipf_centre_point = np.dot(inverse_transformation_matrix, sem_centre_point)

# Normalize the coordinates
ipf_centre_x = int(ipf_centre_point[0, 0] / ipf_centre_point[2, 0])
ipf_centre_y = int(ipf_centre_point[1, 0] / ipf_centre_point[2, 0])
if ipf_centre_x > fe_xmap.shape[1] or ipf_centre_y > fe_xmap.shape[0] or ipf_centre_x < 0 or ipf_centre_y < 0:
    print("Point is outside the EBSD map")
    pass
else:
    [Eu1, Eu2, Eu3] = np.rad2deg(Rotation.to_euler(fe_xmap[int(ipf_centre_y), int(ipf_centre_x)].orientations))[0]
    

In [7]:
%matplotlib qt
fig, ax = plt.subplots()
ax.imshow(dst_warp_blended)
ax.plot(centre_x, centre_y, 'r+', markersize=14)
marker = [[centre_x, centre_y]]
sem_coords = [[centre_x, centre_y]]

destination_point = np.array([[centre_x], [centre_y], [1]])
original_point = np.dot(inverse_transformation_matrix, destination_point)
# Normalize the coordinates
original_x = int(original_point[0, 0] / original_point[2, 0])
original_y = int(original_point[1, 0] / original_point[2, 0])

ipf_coords = [[original_x, original_y]]
euler3 = [list(np.rad2deg(Rotation.to_euler(fe_xmap[int(original_y), int(original_x)].orientations))[0])]

def onclick(event):
    if event.dblclick:
        x, y = int(event.xdata), int(event.ydata)  # Get the pixel coordinates from the click event
        plt.cla()
        
        ax.imshow(dst_warp_blended)
        
        destination_point = np.array([[x], [y], [1]])
        original_point = np.dot(inverse_transformation_matrix, destination_point)
        # Normalize the coordinates
        original_x = int(original_point[0, 0] / original_point[2, 0])
        original_y = int(original_point[1, 0] / original_point[2, 0])
        
        sem_coords.append([x, y])
        ipf_coords.append([original_x, original_y])
        
        
        if original_x > fe_xmap.shape[1] or original_y > fe_xmap.shape[0] or original_x < 0 or original_y < 0:
            print("Point is outside the EBSD map")
            pass
        else:
            [Eu1, Eu2, Eu3] = np.rad2deg(Rotation.to_euler(fe_xmap[int(original_y), int(original_x)].orientations))[0]
        
        marker.append(ax.plot(x, y, 'r+', markersize=14))
        ax.set_title(f"IPF coordinate: {original_x}, {original_y},\nEuler angles: {Eu1:.2f}, {Eu2:.2f}, {Eu3:.2f}")
        print(f"Euler angles: {Eu1:.2f}, {Eu2:.2f}, {Eu3:.2f}")
        
        euler3.append([Eu1, Eu2, Eu3])
        
        plt.draw()
        plt.axis('off')
        # print(len(marker))
        return sem_coords, ipf_coords, euler3

# Connect the mouse click event to the function
cid = fig.canvas.mpl_connect('button_press_event', onclick)

# Display the image and allow interactive selection
plt.show()
plt.draw()
plt.axis('off')



(-0.5, 3071.5, 2187.5, -0.5)

Euler angles: 146.90, 43.52, 8.34


In [8]:
tiltX_corr_angle_op = optimized_correction_coefficients["tiltX_corr_angle"] # positive direction => pattern moves DOWN
tiltY_corr_angle_op = optimized_correction_coefficients["tiltY_corr_angle"] # positive direction => pattern moves LEFT
tiltZ_corr_angle_op = optimized_correction_coefficients["tiltZ_corr_angle"] # positive direction => pattern rotates COUNTER-CLOCKWISE
PCz_op = optimized_correction_coefficients["PCz"] # Larger value => smaller angular range

# get the stage rotation and tilt relative to the reference ECP
# angles used here are according to the testing reference frame rather than the SEM software rotation and tilt
st_rot_angle, st_tilt_angle = stagecomputation.get_relative_stage_pos(si_ref_ecp_fname, fe_ecp_fname)

# convert the stage rotation and tilt to Rotation objects
# the negative sign is used because the stage rotation and tilt active rotations. 
# In EBSD, rotations are usually in passive form according to Bunge convention
st_rot = Rotation.from_axes_angles(axes = [0, 0, 1], 
                                   angles = -st_rot_angle, 
                                   degrees = True)
st_tilt = Rotation.from_axes_angles(axes = [0, 1, 0], 
                                    angles = -st_tilt_angle, 
                                    degrees=True)

fe_xtal_rotation = Rotation.from_euler(np.deg2rad(euler3[-1])) * Rotation.from_axes_angles([0, 0, 1], -np.pi / 2)
ecp_resolution = io.get_sem_metadata(fe_ecp_fname)["resolution"]


sim_RKP = rkp.get_sim_rkp(RKP_masterpattern = fe_MP,
                              xtal_rotation = fe_xtal_rotation,
                              st_rot_angle = st_rot_angle,
                              st_tilt_angle = st_tilt_angle,
                              corr_angles=[tiltX_corr_angle_op, tiltY_corr_angle_op, tiltZ_corr_angle_op],
                              ref_ECP=si_ref_ecp_fname,
                              cam_length=PCz_op,
                              RKP_shape=ecp_resolution)

fig1, axs = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
axs[0,0].imshow(dst_warp_blended)
axs[0,0].plot(sem_coords[-1][0], sem_coords[-1][1], 'r+', markersize=14)
axs[0,0].set_title("SEM/IPF Overview")
axs[0,0].axis("off")

axs[0,1].imshow(fe_exp_sem, cmap="gray")
axs[0,1].set_title("Experimental SEM")
axs[0,1].axis("off")

axs[1,0].imshow(fe_exp_ecp[:ecp_resolution[1],:], cmap='gray')
axs[1,0].plot(ecp_resolution[0]//2, ecp_resolution[1]//2, 'r+', markersize=14)
axs[1,0].set_title("Experimental ECP")
axs[1,0].axis("off")

axs[1,1].imshow(np.squeeze(sim_RKP.data), cmap='gray')
axs[1,1].plot(ecp_resolution[0]//2, ecp_resolution[1]//2, 'r+', markersize=14)
axs[1,1].set_title("Simulated RKP")
axs[1,1].axis("off")

# axs[1,2].imshow(normalize(fe_exp_ecp[:ecp_resolution[1],:]) - normalize(np.squeeze(sim_RKP.data)), cmap='bwr')
# axs[1,2].set_title("Difference")
# axs[1,2].axis("off")

plt.tight_layout()
print(f"stage rotation: {np.round(st_rot_angle, 2)}, stage tilt: {np.round(st_tilt_angle, 2)}")

stage rotation: 0.0, stage tilt: 0.0
